In [1]:
import pandas as pd
from utils import Variable, read_values
import numpy as np
values = read_values()
df = values[2023][1]
df.tail()

,open_time,open,high,low,close,volume,close_time,quote_assets_volume,number_of_trades,taker_buy_base_asset_vol,taker_buy_quote_asset_vol,ignore,open_date,diff,dow
738,1675191600000,23166.54,23188.58,23087.28,23163.88,9139.87470,1675195199999,2.115525e+08,255342,4629.58694,1.071623e+08,0,2023-01-31 19:00:00,-2.66,1
739,1675195200000,23162.99,23172.80,23045.07,23095.41,10546.57529,1675198799999,2.437793e+08,278257,5310.79043,1.227582e+08,0,2023-01-31 20:00:00,-67.58,1
740,1675198800000,23095.41,23124.34,22812.69,22948.69,17969.15727,1675202399999,4.125522e+08,415960,8897.44872,2.042856e+08,0,2023-01-31 21:00:00,-146.72,1
741,1675202400000,22947.66,23320.00,22872.12,23181.81,12479.54407,1675205999999,2.877016e+08,329247,6735.94740,1.553381e+08,0,2023-01-31 22:00:00,234.15,1
742,1675206000000,23181.81,23188.70,23081.19,23125.13,10955.34795,1675209599999,2.533282e+08,290364,5462.81000,1.263245e+08,0,2023-01-31 23:00:00,-56.68,1


In [2]:
import ta
#df.ta.indicators()
#help(ta.atr)


# Compute ATR (Average True Range)
df['ATR'] = ta.volatility.AverageTrueRange(df['high'], df['low'], df['close'], window=20).average_true_range()

# Compute RSI (Relative Strength Index)
df['RSI'] = ta.momentum.RSIIndicator(df['close'], window=14).rsi()

# Compute Midprice
df['Average'] = (df['high'] + df['low']) / 2  # Equivalent to midprice

# Compute SMA (Simple Moving Averages)
df['MA40'] = ta.trend.SMAIndicator(df['close'], window=40).sma_indicator()
df['MA80'] = ta.trend.SMAIndicator(df['close'], window=80).sma_indicator()
df['MA160'] = ta.trend.SMAIndicator(df['close'], window=160).sma_indicator()

from scipy.stats import linregress
def get_slope(array):
    y = np.array(array)
    x = np.arange(len(y))
    slope, intercept, r_value, p_value, std_err = linregress(x,y)
    return slope

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
backrollingN = 6
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
df['slopeMA40'] = df['MA40'].rolling(window=backrollingN).apply(get_slope, raw=True)
df['slopeMA80'] = df['MA80'].rolling(window=backrollingN).apply(get_slope, raw=True)
df['slopeMA160'] = df['MA160'].rolling(window=backrollingN).apply(get_slope, raw=True)
df['AverageSlope'] = df['Average'].rolling(window=backrollingN).apply(get_slope, raw=True)
df['RSISlope'] = df['RSI'].rolling(window=backrollingN).apply(get_slope, raw=True)

In [3]:
#Target flexible way
pipdiff = 500*1e-5 #for TP
SLTPRatio = 2 #pipdiff/Ratio gives SL

def label_trends(barsupfront, df1):
    length = len(df1)
    high = list(df1['high'])
    low = list(df1['low'])
    close = list(df1['close'])
    open = list(df1['open'])
    trendcat = [None] * length

    for line in range(0,length-barsupfront-2):
        valueOpenLow = 0
        valueOpenHigh = 0
        for i in range(1,barsupfront+2):
            value1 = open[line+1]-low[line+i]
            value2 = open[line+1]-high[line+i]
            valueOpenLow = max(value1, valueOpenLow)
            valueOpenHigh = min(value2, valueOpenHigh)

            if ( (valueOpenLow >= pipdiff) and (-valueOpenHigh <= (pipdiff/SLTPRatio)) ):
                trendcat[line] = 1 #-1 downtrend
                break
            elif ( (valueOpenLow <= (pipdiff/SLTPRatio)) and (-valueOpenHigh >= pipdiff) ):
                trendcat[line] = 2 # uptrend
                break
            else:
                trendcat[line] = 0 # no clear trend
            
    return trendcat

,open_time,open,high,low,close,volume,close_time,quote_assets_volume,number_of_trades,taker_buy_base_asset_vol,...,RSI,Average,MA40,MA80,MA160,slopeMA40,slopeMA80,slopeMA160,AverageSlope,RSISlope
0,1672534800000,16529.59,16556.80,16525.78,16551.47,3590.06669,1672538399999,5.937676e+07,126556,1730.24901,...,NaN,16541.290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1672538400000,16551.47,16559.77,16538.14,16548.19,3318.84038,1672541999999,5.491945e+07,115398,1611.12302,...,NaN,16548.955,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1672542000000,16548.19,16548.19,16518.21,16533.04,4242.08050,1672545599999,7.012254e+07,137724,2096.09287,...,NaN,16533.200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1672545600000,16533.04,16535.97,16511.92,16521.85,4285.00909,1672549199999,7.080264e+07,129535,2188.40175,...,NaN,16523.945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1672549200000,16522.22,16534.98,16512.30,16530.71,3717.37178,1672552799999,6.141222e+07,118541,1925.65583,...,NaN,16523.640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
